# Preparação

In [ ]:
import numpy as np
import random
import sqlite3
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
random.seed(2702)
n_populacao = 203062512

# Base de dados

In [ ]:
%load_ext sql

In [ ]:
%%sql
sqlite:////content/drive/MyDrive/brasileiros.db

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/brasileiros.db')
cursor = conn.cursor()

In [ ]:
cursor.execute("create table if not exists estado (id int, estado varchar);")
cursor.execute("create table if not exists genero (id int, genero char);")
cursor.execute("create table if not exists raca (id int, raca varchar);")
cursor.execute("create table if not exists idade (id int, idade int);")
cursor.execute("create table if not exists escolaridade (id int, escolaridade varchar);")
cursor.execute("create table if not exists renda (id int, renda double);")

In [ ]:
r_set=conn.execute('''select count(*) from estado limit 10''')
for row in r_set:
  print("estado", row)

In [ ]:
r_set=conn.execute('''select count(*) from estado''')
for row in r_set:
  print("estado", row)

r_set=conn.execute('''select count(*) from genero''')
for row in r_set:
  print("genero", row)

r_set=conn.execute('''select count(*) from raca''')
for row in r_set:
  print("raca", row)

r_set=conn.execute('''select count(*) from idade''')
for row in r_set:
  print("idade", row)

r_set=conn.execute('''select count(*) from escolaridade''')
for row in r_set:
  print("escolaridade", row)

r_set=conn.execute('''select count(*) from renda''')
for row in r_set:
  print("renda", row)

# Estado

In [ ]:
# 1. Obter população por Estado e instanciar N indivíduos com suas respectivas moradias
# https://g1.globo.com/economia/censo/noticia/2023/06/28/censo-2022-brasil-tem-203-milhoes-de-habitantes-47-milhoes-a-menos-que-estimativa-do-ibge.ghtml

estado = np.array([None] * n_populacao)

# Acre – 830.026 habitantes
estado[0:830026] = 'AC'

# Alagoas – 3.127.511 habitantes
estado[830026:3957537] = 'AL'

# Amapá – 733.508 habitantes
estado[3957537:4691045] = 'AP'

# Amazonas – 3.941.175 habitantes
estado[4691045:8632220] = 'AM'

# Bahia – 14.136.417 habitantes
estado[8632220:22768637] = 'BA'

# Ceará – 8.791.688 habitantes
estado[22768637:31560325] = 'CE'

# Distrito Federal – 2.817.068 habitantes
estado[31560325:34377393] = 'DF'

# Espírito Santo – 3.833.486 habitantes
estado[34377393:38210879] = 'ES'

# Goiás – 7.055.228 habitantes
estado[38210879:45266107] = 'GO'

# Maranhão – 6.775.152 habitantes
estado[45266107:52041259] = 'MA'

# Mato Grosso – 3.658.813 habitantes
estado[52041259:55700072] = 'MT'

# Mato Grosso do Sul – 2.756.700 habitantes
estado[55700072:58456772] = 'MS'

# Minas Gerais – 20.538.718 habitantes
estado[58456772:78995490] = 'MG'

# Pará – 8.116.132 habitantes
estado[78995490:87111622] = 'PA'

# Paraíba – 3.974.495 habitantes
estado[87111622:91086117] = 'PB'

# Paraná – 11.443.208 habitantes
estado[91086117:102529325] = 'PR'

# Pernambuco – 9.058.155 habitantes
estado[102529325:111587480] = 'PE'

# Piauí – 3.269.200 habitantes
estado[111587480:114856680] = 'PI'

# Rio de Janeiro – 16.054.524 habitantes
estado[114856680:130911204] = 'RJ'

# Rio Grande do Norte – 3.302.406 habitantes
estado[130911204:134213610] = 'RN'

# Rio Grande do Sul – 10.880.506 habitantes
estado[134213610:145094116] = 'RS'

# Rondônia – 1.581.016 habitantes
estado[145094116:146675132] = 'RO'

# Roraima – 636.303 habitantes
estado[146675132:147311435] = 'RR'

# Santa Catarina – 7.609.601 habitantes
estado[147311435:154921036] = 'SC'

# São Paulo – 44.420.459 habitantes
estado[154921036:199341495] = 'SP'

# Sergipe – 2.209.558 habitantes
estado[199341495:201551053] = 'SE'

# Tocantins – 1.511.459 habitantes
estado[201551053:203062512] = 'TO'

In [ ]:
i = 1

for e in estado:
  cursor.execute( f"INSERT INTO estado (id, estado) VALUES ({i}, '{e}')" )
  i += 1

  if i % 5000000 == 0:
    print (i/203062512*100, "%")

In [ ]:
conn.commit()
del estado

# Genero

In [ ]:
# 2. Obter % de sexualidade e atualizar os indivíduos
# https://investalk.bb.com.br/noticia/ibge-brasil-tem-6-milhoes-de-mulheres-a-mais-que-homens-942-homens-para-cada-100-mulheres

n_homens = (int) (n_populacao * 0.485)

homens = random.sample(range(1, n_populacao+1), k=n_homens)
homens.sort()

i = 1
for h in homens:

  while i < h:
    cursor.execute( f"INSERT INTO genero (id, genero) VALUES ({i}, 'F')" )
    i += 1

  cursor.execute( f"INSERT INTO genero (id, genero) VALUES ({i}, 'M')" )
  i += 1

  if i % 5000000 == 0:
    print (i/203062512*100, "%")

while i <= n_populacao:
  cursor.execute( f"INSERT INTO genero (id, genero) VALUES ({i}, 'F')" )
  i += 1

In [ ]:
conn.commit()
del homens

# Raça

In [ ]:
# 3. Obter % de raça e atualizar os indivíduos
# https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/18319-cor-ou-raca.html

In [ ]:
def definirRacaEstado(populacaoEstado, idxInicial, p_regiao_b, p_regiao_pr):

  print ("Processando população:", idxInicial)

  n_raca_b = (int) (populacaoEstado * p_regiao_b)
  n_raca_pr = (int) (populacaoEstado * p_regiao_pr)
  n_raca_pa = populacaoEstado - n_raca_b - n_raca_pr

  racaSorteada = 'PA'

  for i in range(idxInicial, idxInicial+populacaoEstado):
    rand = random.random()

    if n_raca_b > 0:
      if n_raca_pr > 0:
        if n_raca_pa > 0:

          # Ainda tem as 3 raças para distribuir
          if rand < p_regiao_b:
            racaSorteada = 'B'
          elif rand < p_regiao_b + p_regiao_pr:
            racaSorteada = 'PR'
          else:
            racaSorteada = 'PA'

        else:
          # Apenas raça B e PR
          if rand < p_regiao_b:
            racaSorteada = 'B'
          else:
            racaSorteada = 'PR'

      else:
          if n_raca_pa > 0:
            # Apenas raça B e PA
            if rand < p_regiao_b:
              racaSorteada = 'B'
            else:
              racaSorteada = 'PA'
          else:
              # Apenas raça B
              racaSorteada = 'B'
    else:
      if n_raca_pr > 0:
        if n_raca_pa > 0:
            # Apenas raça PR e PA
            if rand < p_regiao_pr:
              racaSorteada = 'PR'
            else:
              racaSorteada = 'PA'
        else:
          # Apenas PR
          racaSorteada = 'PR'
      else:
        # Apenas PA
        racaSorteada = 'PA'


    if racaSorteada == 'PR':
      cursor.execute( f"INSERT INTO raca (id, raca) VALUES ({i}, 'PR')" )
      n_raca_pr -= 1
    elif racaSorteada == 'B':
      cursor.execute( f"INSERT INTO raca (id, raca) VALUES ({i}, 'B')" )
      n_raca_b -= 1
    else:
      cursor.execute( f"INSERT INTO raca (id, raca) VALUES ({i}, 'PA')" )
      n_raca_pa -= 1

In [ ]:
# # B - branca
# PR - preta
# PA - parda

# Região        Branca  PReta PArda
# norte         19.7    8.3   70.1
# nordeste      24.9    13.4  60.5
# sudeste       50.1    11.2  37.3
# sul           72.8    5.4   20.9
# centro oeste  36.1    9.6   53

p_norte_b = 0.197
p_norte_pr = 0.083
p_nordeste_b = 0.249
p_nordeste_pr = 0.134
p_sudeste_b = 0.501
p_sudeste_pr = 0.112
p_sul_b = 0.728
p_sul_pr = 0.054
p_centro_b = 0.361
p_centro_pr = 0.096

# Acre – 830.026 habitantes
definirRacaEstado(830026, 1, p_norte_b, p_norte_pr)

# Alagoas – 3.127.511 habitantes
definirRacaEstado(3127511, 830027, p_nordeste_b, p_nordeste_pr)

# Amapá – 733.508 habitantes
definirRacaEstado(733508, 3957538, p_norte_b, p_norte_pr)

# Amazonas – 3.941.175 habitantes
definirRacaEstado(3941175, 4691046, p_norte_b, p_norte_pr)

# Bahia – 14.136.417 habitantes
definirRacaEstado(14136417, 8632221, p_nordeste_b, p_nordeste_pr)

# Ceará – 8.791.688 habitantes
definirRacaEstado(8791688, 22768638, p_nordeste_b, p_nordeste_pr)

# Distrito Federal – 2.817.068 habitantes
definirRacaEstado(2817068, 31560326, p_centro_b, p_centro_pr)

# Espírito Santo – 3.833.486 habitantes
definirRacaEstado(3833486, 34377394, p_sudeste_b, p_sudeste_pr)

# Goiás – 7.055.228 habitantes
definirRacaEstado(7055228, 38210880, p_centro_b, p_centro_pr)

# Maranhão – 6.775.152 habitantes
definirRacaEstado(6775152, 45266108, p_nordeste_b, p_nordeste_pr)

# Mato Grosso – 3.658.813 habitantes
definirRacaEstado(3658813, 52041260, p_centro_b, p_centro_pr)

# Mato Grosso do Sul – 2.756.700 habitantes
definirRacaEstado(2756700, 55700073, p_centro_b, p_centro_pr)

# Minas Gerais – 20.538.718 habitantes
definirRacaEstado(20538718, 58456773, p_sudeste_b, p_sudeste_pr)

# Pará – 8.116.132 habitantes
definirRacaEstado(8116132, 78995491, p_norte_b, p_norte_pr)

# Paraíba – 3.974.495 habitantes
definirRacaEstado(3974495, 87111623, p_nordeste_b, p_nordeste_pr)

# Paraná – 11.443.208 habitantes
definirRacaEstado(11443208, 91086118, p_sul_b, p_sul_pr)

# Pernambuco – 9.058.155 habitantes
definirRacaEstado(9058155, 102529326, p_nordeste_b, p_nordeste_pr)

# Piauí – 3.269.200 habitantes
definirRacaEstado(3269200, 111587481, p_nordeste_b, p_nordeste_pr)

# Rio de Janeiro – 16.054.524 habitantes
definirRacaEstado(16054524, 114856681, p_sudeste_b, p_sudeste_pr)

# Rio Grande do Norte – 3.302.406 habitantes
definirRacaEstado(3302406, 130911205, p_nordeste_b, p_nordeste_pr)

# Rio Grande do Sul – 10.880.506 habitantes
definirRacaEstado(10880506, 134213611, p_sul_b, p_sul_pr)

# Rondônia – 1.581.016 habitantes
definirRacaEstado(1581016, 145094117, p_norte_b, p_norte_pr)

# Roraima – 636.303 habitantes
definirRacaEstado(636303, 146675133, p_norte_b, p_norte_pr)

# Santa Catarina – 7.609.601 habitantes
definirRacaEstado(7609601, 147311436, p_sul_b, p_sul_pr)

# São Paulo – 44.420.459 habitantes
definirRacaEstado(44420459, 154921037, p_sudeste_b, p_sudeste_pr)

# Sergipe – 2.209.558 habitantes
definirRacaEstado(2209558, 199341496, p_nordeste_b, p_nordeste_pr)

# Tocantins – 1.511.459 habitantes
definirRacaEstado(1511459, 201551054, p_norte_b, p_norte_pr)

In [ ]:
conn.commit()

# Idade

In [ ]:
# 4. Obter % das faixas de idade e atualizar os indivíduos sorteando valores da faixa
# (tem que mudar distribuição na última faixa)
# https://biblioteca.ibge.gov.br/visualizacao/livros/liv102004_informativo.pdf

In [ ]:
def definirFaixaEtaria(inicio, fim, porcentagem, ids, count):
  print ("Processando faixa etária:", inicio, fim)

  n_faixa = (int) (n_populacao * porcentagem)

  idades = np.random.randint(inicio, fim+1, n_faixa)
  countIdade = 0

  for n in range(n_faixa):
    id = ids[count]
    idade = idades[countIdade]

    count += 1
    countIdade += 1

    cursor.execute( f"INSERT INTO idade (id, idade) VALUES ({id}, {idade})" )

  del idades
  return count

In [ ]:
ids = np.arange(1, n_populacao+1)
np.random.shuffle(ids)
count = 0

# 0 a 4 anos: 6,8%
count = definirFaixaEtaria(0, 4, 0.068, ids, count)

# 5 a 9 anos: 6,8%
count = definirFaixaEtaria(5, 9, 0.068, ids, count)

# 10 a 13 anos: 5,4%
count = definirFaixaEtaria(10, 13, 0.054, ids, count)

# 14 a 17 anos: 5,6%
count = definirFaixaEtaria(14, 17, 0.056, ids, count)

# 18 a 19 anos: 2,9%
count = definirFaixaEtaria(18, 19, 0.029, ids, count)

# 20 a 24 anos: 7,9%
count = definirFaixaEtaria(20, 24, 0.079, ids, count)

# 25 a 29 anos: 8,0%
count = definirFaixaEtaria(25, 29, 0.08, ids, count)

# 30 a 39 anos: 15,9%
count = definirFaixaEtaria(30, 39, 0.159, ids, count)

# 40 a 49 anos: 14,2%
count = definirFaixaEtaria(40, 49, 0.142, ids, count)

# 50 a 59 anos: 11,4%
count = definirFaixaEtaria(50, 59, 0.114, ids, count)

# 60 a 64 anos: 4,6%
count = definirFaixaEtaria(60, 64, 0.046, ids, count)

# 65 ou mais: 10,5%
print ("Processando faixa etária:", 65)
registros = range(count, n_populacao)
randomGamma = np.random.gamma(2, 1, len(registros))
min_val = 65
max_val = 120
range_out = max_val - min_val
lo_in = np.min(randomGamma)
hi_in = np.max(randomGamma)
range_in = hi_in - lo_in
idades = ((randomGamma - lo_in) / range_in) * range_out + min_val
countIdade = 0
for count in registros:
  id = ids[count]
  idade = round(idades[countIdade])
  countIdade += 1
  cursor.execute( f"INSERT INTO idade (id, idade) VALUES ({id}, {idade})" )

In [ ]:
conn.commit()
del ids
del registros

# Escolaridade

In [ ]:
# 5. Obter % de escolaridade e atualizar indivíduos
# https://sidra.ibge.gov.br/tabela/7267#resultado
# https://biblioteca.ibge.gov.br/visualizacao/livros/liv102002_informativo.pdf

# Média brasileira de nível de instrução considerando sexo
# Homens	  Fundamental incompleto	0,326483773
# Homens	  Fundamental completo	  0,179702349
# Homens	  Médio completo	        0,355627279
# Homens	  Superior completo	      0,1381866
# Mulheres	Fundamental incompleto	0,296716324
# Mulheres	Fundamental completo	  0,161678522
# Mulheres	Médio completo	        0,361014182
# Mulheres	Superior completo	      0,180590972

# FI - fundamental incompleto
# FC - fundamental completo
# MC - médio completo
# SC - superior compelto

In [ ]:
# recuperar sexo em sequência
generos = conn.execute("select id, genero from genero")

escolaridade = ''

for id, genero in generos:
  p = np.random.random()

  if genero == 'M':
    if p < 0.326483773:
      escolaridade = 'FI'
    elif p < 0.506186122:
      escolaridade = 'FC'
    elif p < 0.861813401:
      escolaridade = 'MC'
    else:
      escolaridade = 'SC'

  else:
    if p < 0.296716324:
      escolaridade = 'FI'
    elif p < 0.458394846:
      escolaridade = 'FC'
    elif p < 0.819409028:
      escolaridade = 'MC'
    else:
      escolaridade = 'SC'

  cursor.execute( f"INSERT INTO escolaridade (id, escolaridade) VALUES ({id}, '{escolaridade}')" )

  if id % 5000000 == 0:
    print (id/203062512*100, "%")

In [ ]:
conn.commit()
del generos

# Renda

In [ ]:
# 6. Obter % de renda e atualizar os indivíduos
# https://sidra.ibge.gov.br/tabela/7444#resultado
# https://sidra.ibge.gov.br/tabela/7441#resultado
# https://sidra.ibge.gov.br/tabela/7442#resultado
# https://sidra.ibge.gov.br/tabela/7443#resultado
# https://sidra.ibge.gov.br/tabela/7535#resultado
# https://sidra.ibge.gov.br/tabela/7527#resultado

In [ ]:
# Estado	Homens	Mulheres	Branca	Preta	Parda	FI	FC	MC	SC	14 a 17 anos	18 a 29 anos	30 a 39 anos	40 a 49 anos	50 a 59 anos	60 anos ou mais	Minimo	Máximo	Até o P5	Maior que o P5 até o P10	Maior que o P10 até o P20	Maior que o P20 até o P30	Maior que o P30 até o P40	Maior que o P40 até o P50	Maior que o P50 até o P60	Maior que o P60 até o P70	Maior que o P70 até o P80	Maior que o P80 até o P90	Maior que o P90 até o P95	Maior que o P95 até o P99	Maior que o P99
rendas = {
  'AC': [2240,2186,2730,2014,2116,1269.5,1506,1827.5,4146,681,1598,2302,2524,2781,2320,1596.75,2974.25,257,486,804,1138,1202,1230,1481,1886,2455,3453,5127,8448,20050],
  'AL': [1882,1629,2138,1431,1677,989,1171,1996.5,3799,468,1315,1930,1938,2002,2524,1129.25,2585.75,166,320,532,885,1191,1200,1209,1321,1721,2477,3735,7007,20351],
  'AP': [2365,2354,3455,1967,2121,1226.5,1227,1823.5,4487,499,1592,2107,2775,3123,3375,1511.75,3420.5,246,431,613,990,1199,1218,1450,1881,2414,3733,5658,10432,23469],
  'AM': [2052,1891,2600,1961,1864,968.5,1198,1720,4627,683,1390,2066,2205,2480,2493,1409,2943,181,313,519,845,1112,1205,1292,1586,2081,3022,4997,8650,17335],
  'BA': [1712,1585,2231,1490,1568,943.5,1132.5,1642,3904,654,1231,1718,1951,1817,1805,1342.75,2449.5,103,231,410,650,973,1196,1207,1314,1733,2453,3888,7013,15664],
  'CE': [1877,1674,2300,1939,1555,871.5,1132.5,1641.5,4236,498,1416,1909,1770,2086,2332,1990.75,2686.25,118,236,413,675,1022,1201,1212,1337,1756,2655,4133,7334,21157],
  'DF': [4984,3722,5950,3175,3647,1784.5,1905.5,2650.5,7696,684,2225,4405,5787,5385,5736,3819.25,6104.25,439,831,1176,1275,1544,1947,2571,3409,5069,7845,12022,19737,36640],
  'ES': [3045,2084,3440,1903,2243,1631.5,1695.5,2192,5349,671,1810,2958,2886,2864,3324,1712.5,3789.5,319,621,983,1205,1276,1477,1758,2115,2776,3938,5762,11380,29208],
  'GO': [2936,1998,3146,2064,2249,1500,1831.5,2294.5,4610,841,1768,3104,2669,2745,3058,1600.75,3449,346,706,1122,1212,1301,1529,1836,2146,2694,3647,5435,9565,26540],
  'MA': [1750,1533,2389,1499,1504,888.5,1063,1671.5,3783,457,1195,1733,1763,2084,2374,1138,2574,106,230,391,619,930,1191,1207,1325,1714,2440,3831,6747,18776],
  'MT': [3199,2303,3595,2351,2571,2166.5,2202,2578,4674,1009,2220,3140,3171,3252,3252,1966.25,3680,409,820,1174,1263,1533,1903,2153,2643,3216,4445,5870,9851,25123],
  'MS': [3371,2450,3723,2302,2420,2001.5,1919,2643.5,5347,880,2036,3150,3462,3549,3831,1887.75,4068,363,771,1174,1294,1483,1727,2055,2531,3118,4644,7019,12032,25848],
  'MG': [2642,1912,2825,1844,2031,1350,1797.5,2108,4410,716,1722,2537,2650,2461,2844,1455.5,3180.25,284,581,974,1203,1227,1404,1630,1958,2453,3345,4967,9006,23708],
  'PA': [2007,1743,2805,1727,1719,1093.5,1256.5,1851,4768,580,1344,1975,2137,2505,2259,1473.25,3021.25,140,282,468,724,1051,1201,1251,1534,1967,2752,4324,8556,23010],
  'PB': [2283,1828,2266,1509,2130,866.5,1110,1768,6313,406,1264,1838,3456,1934,2536,1213.25,3579.5,121,233,410,696,1072,1199,1204,1299,1666,2452,4212,7437,54954],
  'PR': [3268,2438,3276,2399,2191,1645,1965.5,2361,5456,882,2158,3085,3243,3432,3436,1789,3859,438,840,1172,1360,1534,1772,2012,2479,3095,4337,6414,11165,28750],
  'PE': [1967,1594,2211,1821,1595,1001,1143,1694.5,4076,546,1344,1903,1949,2135,2302,1184,2639,127,269,480,777,1102,1200,1215,1361,1789,2607,4248,7875,19698],
  'PI': [2067,1747,2683,1908,1683,748.5,1179.5,1554.5,4461,327,1157,2133,2327,2259,2381,2054.5,2898,83,181,325,546,841,1173,1206,1368,2042,3138,5123,9245,20399],
  'RJ': [3477,2868,4159,2150,2506,1590.5,1633.5,2545.5,6369,597,1951,3359,3720,3599,3869,2996,4468.5,355,707,1099,1210,1339,1578,1961,2454,3259,4888,8293,14421,33735],
  'RN': [2395,1951,2786,2221,1809,977.5,1435.5,1976.5,5068,600,1604,2257,2363,2558,3028,2357,3319.25,170,355,625,1022,1200,1208,1282,1542,2073,3138,5195,10143,29751],
  'RS': [3329,2488,3177,1929,2142,1788.5,1997.5,2779,5248,961,2080,3137,3428,3361,3214,2039.25,3795.5,399,801,1175,1354,1546,1833,2087,2545,3186,4521,6585,11375,27471],
  'RO': [2554,1971,2770,1906,2194,1629,1900,2100,4022,854,1851,2449,2624,2576,2815,1590,3040.25,337,662,1012,1205,1288,1507,1756,2067,2643,3502,4940,7687,19955],
  'RR': [2663,2359,3658,1937,2351,1484.5,1525.5,2299.5,4594,708,1884,2686,2900,2502,4468,2399.5,3845.75,413,684,1064,1200,1208,1276,1540,2041,2648,3901,6276,9321,26629],
  'SC': [3408,2536,3219,2260,2354,2093.5,2209.5,2632,5041,1073,2276,3310,3518,3193,3487,2125.25,3796.5,603,1078,1277,1538,1786,1987,2233,2654,3222,4439,6117,10587,26705],
  'SP': [3651,2765,3815,2464,2344,2088,1847,2481,6018,947,2141,3522,3953,3561,3474,1975.75,4359.25,396,833,1164,1337,1555,1839,2106,2652,3318,4756,7602,13689,34056],
  'SE': [2176,1810,2573,1782,1819,1004.5,1275,1624.5,4816,451,1389,2306,2175,2420,2140,1329.5,2996.25,121,252,452,741,1096,1200,1212,1394,1860,2874,4875,10274,25399],
  'TO': [2628,2130,3732,1982,2106,1391.5,1405,1946.5,5538,778,1644,2846,2939,2409,3016,1573.75,3728.5,240,458,757,1126,1204,1245,1464,1789,2319,3479,5392,10738,32694]
}

In [ ]:
def ajustarRendaPorFaixa(renda, dados):
  minimo = dados[15]
  maximo = dados[16]
  faixa = (renda - minimo) / (maximo - minimo) * 100

  if faixa < 5: renda = dados[17]
  elif faixa < 10: renda = dados[18]
  elif faixa < 20: renda = dados[19]
  elif faixa < 30: renda = dados[20]
  elif faixa < 40: renda = dados[21]
  elif faixa < 50: renda = dados[22]
  elif faixa < 60: renda = dados[23]
  elif faixa < 70: renda = dados[24]
  elif faixa < 80: renda = dados[25]
  elif faixa < 90: renda = dados[26]
  elif faixa < 95: renda = dados[27]
  elif faixa < 99: renda = dados[28]
  else: renda = dados[29]

  # Adicionando variação de 10%
  variacao = np.random.random() / 10
  if np.random.random() > 0.5:
    renda += renda*variacao
  else:
    renda -= renda*variacao

  return renda

In [ ]:
def getRenda(dados, idade, genero, raca, escolaridade):
  rendaIdade = 0.0

  if idade < 14:
    return rendaIdade
  elif idade <= 17:
    rendaIdade = dados[9]
  elif idade <= 29:
    rendaIdade = dados[10]
  elif idade <= 39:
    rendaIdade = dados[11]
  elif idade <= 49:
    rendaIdade = dados[12]
  elif idade <= 59:
    rendaIdade = dados[13]
  else:
    rendaIdade = dados[14]

  rendaGenero = 0.0
  if genero == 'M':
    rendaGenero = dados[0]
  else:
    rendaGenero = dados[1]

  rendaRaca = 0.0
  if raca == 'B':
    rendaRaca = dados[2]
  elif raca == 'PR':
    rendaRaca = dados[3]
  else:
    rendaRaca = dados[4]

  rendaEscolaridade = 0.0
  if escolaridade == 'FI':
    rendaEscolaridade = dados[5]
  elif escolaridade == 'FC':
    rendaEscolaridade = dados[6]
  elif escolaridade == 'MC':
    rendaEscolaridade = dados[7]
  else:
    rendaEscolaridade = dados[8]

  renda = (rendaIdade + rendaGenero + rendaRaca + rendaEscolaridade) / 4

  renda = ajustarRendaPorFaixa(renda, dados)

  return round(renda, 2)

In [ ]:
estados = conn.execute("select id, estado from estado order by id")
generos = conn.execute("select genero from genero order by id")
racas = conn.execute("select raca from raca order by id")
escolaridades = conn.execute("select escolaridade from escolaridade order by id")
idades = conn.execute("select idade from idade order by id")

In [ ]:
idx = 0

while idx < n_populacao:

  id_estado = estados.fetchone()
  if id_estado is None:
    break

  dados = rendas[id_estado[1]]

  idade = idades.fetchone()[0]
  genero = generos.fetchone()[0]
  raca = racas.fetchone()[0]
  escolaridade = escolaridades.fetchone()[0]

  renda = getRenda(dados, idade, genero, raca, escolaridade)

  cursor.execute( f"INSERT INTO renda (id, renda) VALUES ({id_estado[0]}, '{renda}')" )

  idx += 1
  if idx % 5000000 == 0:
    print (idx/203062512*100, "%")

In [ ]:
conn.commit()

# Finalizar base de dados

In [ ]:
# Adicionar índices
conn.execute("CREATE UNIQUE INDEX index_estado on estado (id);")
conn.execute("CREATE UNIQUE INDEX index_genero on genero (id);")
conn.execute("CREATE UNIQUE INDEX index_raca on raca (id);")
conn.execute("CREATE UNIQUE INDEX index_idade on idade (id);")
conn.execute("CREATE UNIQUE INDEX index_escolaridade on escolaridade (id);")
conn.execute("CREATE UNIQUE INDEX index_renda on renda (id);")

In [ ]:
# Juntar tabelas
conn.execute('''CREATE TABLE brasileiros2022 AS
SELECT estado.id, estado, genero, raca, idade, escolaridade, renda
  FROM estado
  INNER JOIN genero ON estado.id = genero.id
  INNER JOIN raca ON estado.id = raca.id
  INNER JOIN idade ON estado.id = idade.id
  INNER JOIN escolaridade ON estado.id = escolaridade.id
  INNER JOIN renda ON estado.id = renda.id;''')

In [ ]:
# Ajustar escolaridade de acordo com a idade
conn.execute("UPDATE brasileiros2022 SET escolaridade = 'MC' WHERE escolaridade = 'SC' and idade >= 14 and idade <= 22;")

conn.execute("UPDATE brasileiros2022 SET escolaridade = 'FC' WHERE escolaridade = 'MC' and idade >= 14 and idade <= 17;")

conn.execute("UPDATE brasileiros2022 SET escolaridade = 'FI' WHERE escolaridade = 'FC' and idade = 15;")

conn.execute("UPDATE brasileiros2022 SET escolaridade = 'FI' WHERE idade < 14;")

In [ ]:
conn.commit()

In [ ]:
r_set=conn.execute('''select count(*) from brasileiros2022 limit 100''')
for row in r_set:
  print(row)